#### Voom Normalization of Exp Data

In [124]:
import numpy as np 
import pandas as pd 

In [24]:
data = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/DepMap/OmicsExpressionRNASeQCGeneCountProfile.csv")

In [25]:
expr_df = data.rename(columns={"Unnamed: 0": "ProfileID"})

In [26]:
profile = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/DepMap/OmicsProfiles.csv")

In [27]:
metadata = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/DepMap/Model.csv")

In [28]:
# Step 1: Merge profile → metadata to get ProfileID → StrippedCellLineName
merged_df = profile[['ProfileID', 'ModelID']].merge(
    metadata[['ModelID', 'StrippedCellLineName']],
    on='ModelID', how='left'
)

# Step 2: Create a mapping dictionary
profile_to_stripped = dict(zip(merged_df['ProfileID'], merged_df['StrippedCellLineName']))

# Step 3: Map ProfileID in expression data to StrippedCellLineName
expr_df['StrippedCellLineName'] = expr_df['ProfileID'].map(profile_to_stripped)

# Optional: Set it as index
expr_df = expr_df.set_index('StrippedCellLineName')

# Optional: Drop old ProfileID column
expr_df = expr_df.drop(columns=['ProfileID'])

In [29]:
expr_df = expr_df[~expr_df.index.duplicated(keep='first')]


In [30]:
import pandas as pd
import re

# Assume df is your DataFrame
expr_df.columns = expr_df.columns.str.replace(r"\s*\(.*?\)", "", regex=True)


In [31]:
import pandas as pd

# Assuming expr_df already has StrippedCellLineName as index
# and shape like (1690, 53969)
design_df = pd.DataFrame({
    'Target': ['cancer'] * expr_df.shape[0]
}, index=expr_df.index)


In [32]:
expr_df

,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,AL627309.1,...,ERCC-00157,ERCC-00158,ERCC-00160,ERCC-00162,ERCC-00163,ERCC-00164,ERCC-00165,ERCC-00168,ERCC-00170,ERCC-00171
StrippedCellLineName,,,,,,,,,,,,,,,,,,,,,
LC1SQSF,0.0,193.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,4.0,88.0,16.0,0.0,26.0,2.0,16.0,2177.0
COGAR359,0.0,108.0,0.0,4.0,1.0,0.0,0.0,1.0,2.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COLO794,0.0,179.0,0.0,1.0,0.0,1.0,2.0,3.0,3.0,8.0,...,20.0,2.0,24.0,357.0,67.0,0.0,127.0,4.0,65.0,11630.0
KE97,1.0,141.0,0.0,1.0,0.0,9.0,1.0,0.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BFTC909,1.0,115.0,0.0,0.0,0.0,1.0,5.0,5.0,11.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CI34601,0.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,...,45.0,0.0,45.0,266.0,74.0,0.0,251.0,1.0,71.0,13498.0
ABMT0558,11.0,356.0,0.0,1.0,0.0,2.0,1.0,1.0,3.0,20.0,...,6.0,0.0,12.0,112.0,6.0,0.0,74.0,2.0,20.0,3368.0
42MGBA,1.0,236.0,0.0,2.0,0.0,0.0,1.0,1.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Transpose the DataFrame
expr_df_t = expr_df.transpose()

# Set the gene names as the index (they are already the index after transpose)
expr_df_t.index.name = "Gene"  # Optional: name the index for clarity

# Reset column names if needed
expr_df_t.columns.name = None  # Remove any column grouping names

# View the result
expr_df_t.head()


,LC1SQSF,COGAR359,COLO794,KE97,BFTC909,KCIMOH1,KKU213,RT4,SNU283,YKG1,...,WM4235,ACHN,COLO792,ECC2,A673,CI34601,ABMT0558,42MGBA,NCIH1436,LU65
Gene,,,,,,,,,,,,,,,,,,,,,
DDX11L1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,...,0.0,1.0,1.0,5.0,8.0,0.0,11.0,1.0,1.0,0.0
WASH7P,193.0,108.0,179.0,141.0,115.0,255.0,152.0,316.0,193.0,171.0,...,200.0,109.0,186.0,1375.0,499.0,131.0,356.0,236.0,230.0,152.0
MIR6859-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
MIR1302-2HG,0.0,4.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,...,0.0,3.0,2.0,8.0,0.0,0.0,1.0,2.0,0.0,0.0
MIR1302-2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
import sys
import click
import numpy as np
import pandas as pd
import rpy2.robjects as ro
#from rpy2.rinterface import RRuntimeError
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from statsmodels.stats.multitest import multipletests
from rpy2.robjects import pandas2ri, default_converter

# Load your data
# Make sure your expression_file.xlsx has an 'ID' column to be set as the index
data = expr_df_t

# Load your design file
design = design_df

# Import R libraries
# Make sure you have installed these R packages and specified the correct library path if necessary
base = importr('base')
stats = importr('stats')
limma = importr('limma') # No need for lib_loc if packages are in the default R library path

# Convert data and design pandas dataframes to R dataframes
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)
    r_design = ro.conversion.py2rpy(design)

    # Use the genes index column from data as an R String Vector
    genes = ro.StrVector(data.index.tolist())

# Create a model matrix
# Since there is only one target, we create a design matrix that fits an intercept for this group.
form = Formula('~ 1')
r_design = stats.model_matrix(form, data=r_design)

# Perform voom normalization
voom_output = limma.voom(r_data, r_design, plot=False)

# The voom_output object contains the normalized expression values in the 'E' component.
# You can access it and convert it back to a pandas DataFrame.
from rpy2.robjects.conversion import rpy2py

with localconverter(default_converter + pandas2ri.converter):
    normalized_expression_matrix = rpy2py(voom_output.rx2('E'))


# Create a pandas DataFrame with the normalized data
normalized_df = pd.DataFrame(normalized_expression_matrix, index=data.index, columns=data.columns)

# You can now work with the normalized_df DataFrame
print("Voom normalized data:")
print(normalized_df.head())


/home/nilabjab/miniconda3/envs/pyenv/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


Voom normalized data:
              LC1SQSF  COGAR359   COLO794      KE97   BFTC909   KCIMOH1  \
Gene                                                                      
DDX11L1     -7.691625 -7.599857 -8.266256 -6.386963 -6.769445 -8.154407   
WASH7P       0.904565  0.161694  0.221584  0.172733 -0.502658  0.842773   
MIR6859-1   -7.691625 -7.599857 -8.266256 -7.971926 -8.354407 -8.154407   
MIR1302-2HG -7.691625 -4.429932 -6.681294 -6.386963 -8.354407 -5.832479   
MIR1302-2   -7.691625 -6.014895 -8.266256 -7.971926 -8.354407 -8.154407   

               KKU213       RT4    SNU283      YKG1  ...    WM4235      ACHN  \
Gene                                                 ...                       
DDX11L1     -6.432840 -5.530411 -6.532509 -5.649825  ... -7.650880 -6.385505   
WASH7P       0.234863  0.968296  0.478718  0.450312  ...  0.996578 -0.195681   
MIR6859-1   -8.017802 -6.752803 -8.117472 -7.971753  ... -7.650880 -7.970468   
MIR1302-2HG -8.017802 -8.337766 -6.532509 -6.386790 

In [36]:
normalized_df.shape

(57975, 1517)

In [58]:
# Transpose the DataFrame
transposed_df = normalized_df.T

# Optionally reset the index and rename
transposed_df.index.name = 'StrippedCellLineName'
transposed_df.columns.name = None


#### Subsetting same rows between expression and dependency

In [59]:
dep = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/DepMap/CRISPRGeneDependency.csv")

In [60]:
dep = dep.rename(columns={"Unnamed: 0": "ModelID"})

In [61]:
# Step 2: Create a mapping dictionary
model_to_stripped = dict(zip(merged_df['ModelID'], merged_df['StrippedCellLineName']))

In [62]:
# Step 3: Map ProfileID in expression data to StrippedCellLineName
dep['StrippedCellLineName'] = dep['ModelID'].map(model_to_stripped)

# Optional: Set it as index
dep = dep.set_index('StrippedCellLineName')

# Optional: Drop old ProfileID column
dep = dep.drop(columns=['ModelID'])

In [63]:
import re

# Remove anything in parentheses from column names
dep.columns = dep.columns.to_series().apply(lambda x: re.sub(r"\s*\(.*?\)", "", x))


In [64]:
# Step 2: Find common StrippedCellLineName values
common_cells = dep.index.intersection(transposed_df.index)

# Step 3: Subset and sort both dataframes
dep_subset = dep.loc[common_cells].sort_index()
expr_subset = transposed_df.loc[common_cells].sort_index()

# Optional: Reset index if needed
# dep_subset = dep_subset.reset_index()
# expr_subset = expr_subset.reset_index()


In [65]:
len(common_cells)

1076

#### Common drugs between Creammist and |Opentargets + TTD|

In [68]:
opentarget_df = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/Open_target_drugs.tsv", sep = "\t")

In [69]:

drug_symbol_df = opentarget_df[['drugName', 'symbol']]

# Drop rows where the (drugName, symbol) pair is duplicated (i.e., keep only those that appear once)
unique_pairs_df = drug_symbol_df.drop_duplicates(keep=False)

# Optional: Reset index
unique_pairs_ot = unique_pairs_df.reset_index(drop=True)


In [70]:
# Group by drugName and join all symbols into a comma-separated string
aggregated_df_ot = drug_symbol_df.groupby('drugName')['symbol'].unique().reset_index()

# Convert list of symbols to comma-separated string (optional)
aggregated_df_ot['symbol'] = aggregated_df_ot['symbol'].apply(lambda x: ','.join(sorted(set(x))))


In [71]:
ttd = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/Drug targets.csv")

In [72]:
drug_symbol_df = ttd[['drug_name', 'gene_name']]

# Drop rows where the (drugName, symbol) pair is duplicated (i.e., keep only those that appear once)
unique_pairs_df = drug_symbol_df.drop_duplicates(keep=False)

# Optional: Reset index
unique_pairs_ttd = unique_pairs_df.reset_index(drop=True)

In [73]:
# Group by drugName and join all symbols into a comma-separated string
aggregated_df_ttd = drug_symbol_df.groupby('drug_name')['gene_name'].unique().reset_index()

# Convert list of symbols to comma-separated string (optional)
aggregated_df_ttd['gene_name'] = aggregated_df_ttd['gene_name'].apply(lambda x: ','.join(sorted(set(x))))


In [74]:
aggregated_df_ttd['drug_name'] = aggregated_df_ttd['drug_name'].str.upper()

In [75]:
# Rename columns for consistency
aggregated_df_ot = aggregated_df_ot.rename(columns={"drugName": "Drug", "symbol": "Target"})
aggregated_df_ttd = aggregated_df_ttd.rename(columns={"drug_name": "Drug", "gene_name": "Target"})

# Concatenate the two DataFrames
merged_df = pd.concat([aggregated_df_ot, aggregated_df_ttd], ignore_index=True)


In [76]:
merged_df.shape

(2894, 2)

In [120]:
import requests
import pandas as pd

# API endpoint
url = "https://creammist.mtms.dev/api/drugs"

# Send GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    drug_df = pd.DataFrame(data)  # Convert to pandas DataFrame
    print(drug_df.head())  # Preview the data
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


                         drug_name synonyms target  \
0  (-)-gallocatechin-3-monogallate                   
1                (5Z)-7-Oxozeaenol            TAK1   
2           1-methyl-DL-tryptophan            IDO1   
3                          10-DEBC             AKT   
4                           123138                   

                                      pathway ccle_drug_name  \
0                             natural product                  
1                              Other, kinases                  
2  inhibitor of indoleamine 2,3-dioxygenase 1                  
3                            inhibitor of AKT                  
4                                Unclassified                  

                   ctrp1_drug_name ctrp2_drug_name    gdsc1_drug_name  \
0  (-)-gallocatechin-3-monogallate                                      
1                                                   (5Z)-7-Oxozeaenol   
2           1-methyl-DL-tryptophan                                     

In [122]:
# Convert 'drug_name' column to uppercase and store as a list
uppercase_drug_names = drug_df['drug_name'].str.upper().tolist()

In [126]:
# Convert drug names from both dataframes to uppercase sets
ttd_drugs = set(aggregated_df_ttd['drug_name'].str.upper())
ot_drugs = set(aggregated_df_ot['drugName'].str.upper())

# Combine them
combined_drugs = ttd_drugs.union(ot_drugs)

# Convert your existing list to a set for fast intersection
uppercase_drug_names_set = set(uppercase_drug_names)

# Find common drugs
common_drugs = combined_drugs.intersection(uppercase_drug_names_set)

# Output result
print(f"Number of common drugs: {len(common_drugs)}")
print("Some common drugs:", list(common_drugs)[:10])  # Show first 10


Number of common drugs: 184
Some common drugs: ['SILDENAFIL', 'TANDUTINIB', 'GSK2636771', 'MARAVIROC', 'METHOTREXATE', 'TRAMETINIB', 'TASELISIB', 'FEDRATINIB', 'CPI-613', 'BIRINAPANT']


#### Collect all the Drug IC50 data using Creammist API

In [159]:
import pandas as pd
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm # A great library for progress bars!

# --- Step 1: Define your list of drugs ---
# Let's use a larger list to see the speed-up

# API base URL
BASE_URL = "https://creammist.mtms.dev/api"

# --- Step 2: First, get all the work we need to do ---
# We will create a list of all (drug, cell_id) pairs to query
jobs_to_do = []
print("Phase 1: Gathering all drug/cell line combinations to query...")

# Use tqdm for a nice progress bar
for drug in tqdm(common_drugs, desc="Querying Drugs"):
    cell_lines_url = f"{BASE_URL}/drugs/{drug}"
    try:
        response_cells = requests.get(cell_lines_url)
        response_cells.raise_for_status()
        cell_lines_data = response_cells.json()
        for item in cell_lines_data:
            # Add a tuple of (drug, cell_id) to our list of jobs
            jobs_to_do.append((drug, item['cellosaurus_id']))
    except requests.exceptions.RequestException:
        # Silently skip drugs that fail to be queried
        continue

# Remove duplicate jobs, just in case
jobs_to_do = list(set(jobs_to_do))
print(f"Found a total of {len(jobs_to_do)} unique experiments to fetch.")

# --- Step 3: Define the function that each thread will run ---
# This function fetches the IC50 for a single (drug, cell_id) pair.
def fetch_ic50(drug, cell_id):
    """Fetches a single IC50 value and handles errors."""
    experiment_url = f"{BASE_URL}/experiments/{cell_id}/{drug}"
    try:
        response_exp = requests.get(experiment_url, timeout=10) # Add a timeout
        response_exp.raise_for_status()
        experiment_data = response_exp.json()

        if experiment_data and 'IC50' in experiment_data[0]:
            return {
                'drug': drug,
                'cell_line': cell_id,
                'IC50': experiment_data[0]['IC50']
            }
    except requests.exceptions.RequestException:
        # If there's any error (not found, timeout, etc.), return None
        return None
    return None

# --- Step 4: Run the jobs concurrently using ThreadPoolExecutor ---
results_data = []
# Set the number of concurrent workers. Start with 10-20 and see.
# Don't set this too high!
MAX_WORKERS = 20

print(f"\nPhase 2: Fetching IC50 data with {MAX_WORKERS} concurrent workers...")
# Use a `with` statement for clean management of threads
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submit all jobs to the pool. executor.submit returns a "future" object.
    future_to_job = {executor.submit(fetch_ic50, drug, cell_id): (drug, cell_id) for drug, cell_id in jobs_to_do}

    # Process the results as they are completed using tqdm for a progress bar
    for future in tqdm(as_completed(future_to_job), total=len(jobs_to_do), desc="Fetching Experiments"):
        result = future.result()
        if result is not None:
            # Append the successful result to our list
            results_data.append(result)


print("\nData fetching complete. Creating the final DataFrame...")

# --- Step 5: Create and format the final DataFrame ---
if not results_data:
    print("No data was fetched. The final DataFrame is empty.")
    final_df = pd.DataFrame()
else:
    long_format_df = pd.DataFrame(results_data)
    final_df = long_format_df.pivot(index='drug', columns='cell_line', values='IC50')

# --- Step 6: Display the result ---
print("\nFinal IC50 DataFrame:")
print(final_df)

Phase 1: Gathering all drug/cell line combinations to query...


Querying Drugs: 100%|██████████| 184/184 [01:10<00:00,  2.60it/s]


Found a total of 169933 unique experiments to fetch.

Phase 2: Fetching IC50 data with 20 concurrent workers...


Fetching Experiments: 100%|██████████| 169933/169933 [53:48<00:00, 52.64it/s]  



Data fetching complete. Creating the final DataFrame...

Final IC50 DataFrame:
cell_line    1321N1     2004    22RV1  23132-87      253J  253J-BV  42-MG-BA  \
drug                                                                           
ABIRATERONE     NaN      NaN      NaN       NaN       NaN      NaN       NaN   
ABT-751         NaN      NaN      NaN       NaN       NaN      NaN       NaN   
AFATINIB        NaN  1.47000  2.80303   1.54454  0.285270  1.44631   2.59555   
AFURESERTIB     NaN      NaN  2.11144   1.40029       NaN      NaN   1.29770   
ALECTINIB       NaN      NaN  4.80222   3.94753       NaN      NaN   4.80889   
...             ...      ...      ...       ...       ...      ...       ...   
VISMODEGIB      NaN      NaN  4.58268   5.42715       NaN      NaN   5.72101   
VORINOSTAT      NaN  2.77028  1.08685   1.26483  0.940899  2.42809   2.67771   
VOXTALISIB      NaN      NaN  5.41142   4.18544       NaN      NaN   6.11620   
ZIBOTENTAN      NaN      NaN  7.35143   

In [163]:
final_df.to_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Datasets/Creammist_common_ic50.csv", index=True, header=True)

#### Subset Expr. based on EnrichrKG Genes

In [1]:
# Read the gene names from the saved text file
with open("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Approach3_Latent_factor/git_repo/Data/gene_names.txt", "r") as f:
    enrichr_genes = [line.strip() for line in f]



In [3]:
len(enrichr_genes)

26518

In [80]:
# Only keep genes that are actually in the DataFrame's columns
available_genes = [gene for gene in enrichr_genes if gene in expr_subset.columns]

# Subset the DataFrame to keep only those columns
subset_expr_df = expr_subset[available_genes]

# Optional: if StrippedCellLineName is the index or a column you want to keep:
#subset_expr_df.insert(0, 'Unnamed: 0', transposed_df.index if transposed_df.index.name == 'Unnamed: 0' else transposed_df['Unnamed: 0'])


#### Top 100 HCGs based on Spearmann Corr

In [86]:
final_df.columns = final_df.columns.str.replace('-', '')

In [160]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def generate_drug_hcg_data(expr_path, dependency_path, drug_target_path, ic50_path):
    """
    Generates a list of highly correlated genes (HCG) for each drug based on
    expression, dependency, and drug sensitivity (IC50) data.

    Args:
        expr_path (str): Path to the expression data CSV file.
        dependency_path (str): Path to the dependency data CSV file.
        drug_target_path (str): Path to the drug-target mapping CSV file.
        ic50_path (str): Path to the drug IC50 data CSV file.

    Returns:
        pandas.DataFrame: A DataFrame with 'Drug' and a comma-separated 'HCG_list'.
    """
    print("Loading data files...")
    # Load data, setting the cell line names as the index
    expr_data = pd.read_csv(expr_path, index_col=0)
    dependency_data = pd.read_csv(dependency_path, index_col=0)
    drug_target_data = pd.read_csv(drug_target_path)
    # For IC50 data, drugs are rows, so we set the 'drug' column as index
    drug_ic50_data = pd.read_csv(ic50_path, index_col='drug')

    print("Aligning datasets...")
    # Find common cell lines and genes to work with
    common_cell_lines = sorted(expr_data.index.intersection(dependency_data.index))

    

    # Filter data to only common identifiers
    expr_data = expr_data.loc[common_cell_lines]
    dependency_data = dependency_data.loc[common_cell_lines]

    # --- Main processing loop ---
    drug_hcg_results = {}
    min_ic50_datapoints = 20 # Minimum number of cell lines with IC50 data to run correlation

    # Use tqdm for a progress bar
    for drug in tqdm(drug_ic50_data.index, desc="Processing Drugs"):
        highly_correlated_genes = set()

        # --- Step 1: Get Target-based Correlated Genes (Dependency vs. Expression) ---
        target_info = drug_target_data[drug_target_data['Drug'] == drug]
        if target_info.empty or pd.isna(target_info['Target'].iloc[0]):
            continue # Skip if drug has no targets listed

        # Get the list of targets for the current drug
        drug_targets = [t.strip() for t in target_info['Target'].iloc[0].split(',')]

        for target_gene in drug_targets:
            if target_gene not in dependency_data.columns:
                continue # Skip if target is not in our dependency data

            # Get the dependency series for the target gene
            dependency_series = dependency_data[target_gene]

            # Calculate Spearman correlation between the target's dependency and all gene expressions
            # .corrwith() is efficient and handles alignment automatically
            from scipy.stats import spearmanr

            def fast_spearman_vector(df, series):
                # Drop constant columns (std == 0)
                variable_cols = df.loc[:, df.std() != 0]

                # Rank transform
                df_ranked = variable_cols.rank(axis=0)
                series_ranked = series.rank()

                # Compute Pearson on ranked data (Spearman)
                return df_ranked.corrwith(series_ranked, method='pearson')


            corrs = fast_spearman_vector(expr_data, dependency_series)
            

            # Get the top 100 genes with the highest absolute correlation
            top_100_dep_genes = corrs.abs().nlargest(100).index.tolist()
            highly_correlated_genes.update(top_100_dep_genes)


        # --- Step 2: Get IC50-based Correlated Genes (IC50 vs. Expression) ---
        ic50_series = drug_ic50_data.loc[drug].dropna()

        # Ensure we have enough data points to calculate meaningful correlations
        if len(ic50_series) < min_ic50_datapoints:
            continue

        # Align expression data to only the cell lines where we have IC50 data
        common_ic50_cell_lines = expr_data.index.intersection(ic50_series.index)
        expr_filtered_for_ic50 = expr_data.loc[common_ic50_cell_lines]
        ic50_series_aligned = ic50_series.loc[common_ic50_cell_lines]
        
        corrs_ic50 = fast_spearman_vector(expr_filtered_for_ic50, ic50_series_aligned)
        
        # Get the top 100 genes with the highest absolute correlation
        top_100_ic50_genes = corrs_ic50.abs().nlargest(100).index.tolist()
        highly_correlated_genes.update(top_100_ic50_genes)

        # --- Step 3: Finalize Gene Set ---
        # Add the original drug targets to the set to ensure they are included
        highly_correlated_genes.update(drug_targets)

        # Store the final, sorted list of unique genes
        if highly_correlated_genes:
             drug_hcg_results[drug] = sorted(list(highly_correlated_genes))

    print("Formatting final output...")
    # Convert the results dictionary to a DataFrame
    final_df = pd.DataFrame(list(drug_hcg_results.items()), columns=['Drug', 'HCG_list'])
    
    # Convert the list of genes into a comma-separated string for easier storage
    final_df['HCG_list'] = final_df['HCG_list'].apply(lambda x: ','.join(x))
    
    return final_df


if __name__ == '__main__':
    # --- PLEASE PROVIDE THE CORRECT PATHS TO YOUR FILES HERE ---
    EXPRESSION_FILE = '/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Approach3_Latent_factor/git_repo/Data/Exp.csv'
    DEPENDENCY_FILE = '/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Approach3_Latent_factor/git_repo/Data/Dep.csv'
    DRUG_TARGET_FILE = '/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Approach3_Latent_factor/git_repo/Data/Drug_target_data.csv'
    DRUG_IC50_FILE = '/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Approach3_Latent_factor/git_repo/Data/Creammist_common_ic50.csv'

    # Generate the HCG data
    drug_hcg_df = generate_drug_hcg_data(
        expr_path=EXPRESSION_FILE,
        dependency_path=DEPENDENCY_FILE,
        drug_target_path=DRUG_TARGET_FILE,
        ic50_path=DRUG_IC50_FILE
    )

    # Save the final result to a new CSV file
    #drug_hcg_df.to_csv('drug_hcg_data.csv', index=False)
    #print("\nSuccessfully saved the results to 'drug_hcg_data.csv'")

Loading data files...
Aligning datasets...


Processing Drugs: 100%|██████████| 184/184 [1:20:17<00:00, 26.18s/it]

Formatting final output...


In [89]:
# Assuming your DataFrame is called `drug_hcg_df`
drug_hcg_df['HCG_count'] = drug_hcg_df['HCG_list'].apply(lambda x: len(x.split(',')))

In [127]:
drug_hcg_df

,Drug,HCG_list,HCG_count
0,ABIRATERONE,"ABCG4,ABHD10,ABLIM3,ACO2,ACOT4,ACSBG1,ACSL1,AC...",501
1,ABT-751,"ABALON,ABLIM3,ACOT12,ACR,ACTBP13,ACVR1,ADAMTSL...",807
2,AFATINIB,"AATBC,ACE2,ADAMTS15,ADCY4,ADGRF1,ADGRF4,AHDC1,...",325
3,AFURESERTIB,"AADAT,ABALON,ABCB1,ABCC9,ABI2,ABT1,ACP1,ADGRA3...",397
4,ALECTINIB,"ABCC3,ABI1,ACD,ACKR4,ADAM23,ADCK5,ADGRG2,ADPRM...",402
...,...,...,...
179,VISMODEGIB,"AADAC,ADAM22,ADCY2,ADIRF,AGAP13P,ANK2,ANKS1B,A...",200
180,VORINOSTAT,"ABI3BP,ACOT11,ACSS2,ADAM17,ADAMTS16,ADAMTS5,AD...",500
181,VOXTALISIB,"ABHD17B,ACAP1,ACTBP13,ACTG1P21,ACTL9,ACTR3B,AC...",918
182,ZIBOTENTAN,"A2ML1,ABLIM3,ABTB1,ACVR2B,ADGRG2,ADORA1,ADSSL1...",199


#### Common cell lines for each drug 

In [210]:
# Transpose ic50 so rows = cell lines, columns = drugs
ic50_df = final_df.T

# Ensure both datasets have matching cell lines
common_cell_lines = ic50_df.index.intersection(subset_expr_df.index)

# Filter IC50 data to only those in expression data
ic50_df = ic50_df.loc[common_cell_lines]

# Prepare result list
result = []

for drug in ic50_df.columns:
    non_nan_cell_lines = ic50_df[drug].dropna().index
    sorted_cell_lines = sorted(non_nan_cell_lines)
    result.append({
        'Drug': drug,
        'Cell lines': ','.join(sorted_cell_lines),
        'Number of cell lines': len(sorted_cell_lines)
    })

# Create the final DataFrame
drug_cell_line_df = pd.DataFrame(result)

